## What is RAG ?

RAG is a technique where a language model first looks up (retrieves) useful information from a database or documents, and then uses that information to give a better answer.

## Why and when we prefer RAG over finetuning ?

We **prefer RAG over finetuning** when: We want the model to give **up-to-date or specific answers** from **our own data** without changing the model itself.

---

**Why prefer RAG?**

* **Cheaper & faster** – No need to train the model again.
* **Easier to update** – Just change the documents, not the model.
* **Better for private or large data** – You keep data separate and safe.

---

**When to use RAG?**

* When your data **changes often** (like news, product lists).
* When you want the model to **answer from your documents**.
* When **training a model is too costly or slow**.

---

Think of RAG like **giving the model a library to read** instead of teaching it everything from scratch.


# Install Libraries

* **`langchain`** – Core framework to build LLM-powered applications.
* **`langchain-community`** – Extra integrations like tools, APIs, and vector stores.
* **`langchain-pinecone`** – Connects LangChain with Pinecone for vector storage and retrieval.
* **`langchain_groq`** – Enables LangChain to use Groq's ultra-fast language models.
* **`datasets`** – Provides ready-to-use NLP/ML datasets from Hugging Face.

In [1]:
%pip install langchain==0.3.23 langchain-community==0.3.21 langchain-pinecone==0.2.5 langchain_groq datasets==3.5.0

     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.0 MB 2.0 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.0 MB 2.6 MB/s eta 0:00:01
   --------------------- ------------------ 0.5/1.0 MB 3.7 MB/s eta 0:00:01
   --------------------------------- ------ 0.8/1.0 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.5 MB 8.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.5 MB 7.5 MB/s eta 0:00:01
   ------------------ --------------------- 1.2/2.5 MB 8.5 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 8.3 MB/s eta 0:00:01
   -----------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\shama\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Load API Keys from .env file

### What is env file?

* A .env file is a simple text file that stores environment variables (like API keys and secrets) in key=value format.
* Example content of a .env file:
* PINECONE_API_KEY=your_pinecone_api_key_here
* GROQ_API_KEY=your_groq_api_key_here

* It helps keep sensitive information out of your code and makes it easier to manage secrets securely.

**Imports tools** to:

* Use environment variables (`os`)
* Load values from a `.env` file (`load_dotenv`)
* **os** is a Python built-in module that lets your code interact with the operating system (like Windows, macOS, Linux).
---
* **Loads the `.env` file** so Python can use the secret keys stored in it (like API keys).
* **Gets the values** of `PINECONE_API_KEY` and `GROQ_API_KEY` from the `.env` file.
* **In Short:** This code **reads your secret keys from a `.env` file** so you don’t have to write them directly in your code.


In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the keys
pinecone_key = os.getenv("PINECONE_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

# Check if keys are loaded properly
if pinecone_key:
    print("✅ Pinecone API Key Loaded Successfully")
else:
    print("❌ Pinecone API Key NOT Loaded")

if groq_key:
    print("✅ Groq API Key Loaded Successfully")
else:
    print("❌ Groq API Key NOT Loaded")


✅ Pinecone API Key Loaded Successfully
✅ Groq API Key Loaded Successfully


## What is `langchain_groq`?

`langchain_groq` is a **LangChain integration** that lets you **connect to Groq’s LLMs** (like LLaMA3) easily.

Think of it as a **bridge between LangChain and Groq’s fast language models**.

---

## What is `ChatGroq`?

`ChatGroq` is a **class (tool)** inside `langchain_groq`.

It lets you:

* **Send prompts** to Groq-hosted models
* **Receive responses** from those models
* Use these models in your **LangChain app**, like chatbots, RAG, agents, etc.

---

**Why do we use this?**

Instead of manually setting up HTTP requests to Groq’s API, `ChatGroq` makes it **super easy**:

* Lets you talk to a specific Groq model (`llama3-8b-8192`)
* Works smoothly with LangChain tools (retrievers, chains, memory, etc.)
* Connects securely with your `groq_api_key`

---

#### In Simple Words

* `langchain_groq` lets LangChain talk to Groq.
* `ChatGroq` is the tool that helps you **chat with Groq’s AI model** using your API key.


In [20]:
from langchain_groq import ChatGroq

chat = ChatGroq(
    groq_api_key=groq_key,
    model_name="llama-3.1-8b-instant"  # Correct model name used by Groq
)

Groq uses the **same chat structure as OpenAI** because it runs **OpenAI-compatible models** like `llama3`, `mixtral`, etc.
So just like OpenAI, chats with Groq **typically look like this in plain text**:

```
System: You are a helpful assistant.
User: Hi, how are you?
Assistant: I'm doing well! How can I assist you today?
User: What is quantum computing?
Assistant:
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

---

**In Code (OpenAI/Groq-compatible format):**

When using the API (like with `ChatGroq` or `ChatOpenAI` in LangChain), you use this structure:

```
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "I'm doing well! How can I assist you today?"},
    {"role": "user", "content": "What is quantum computing?"}
]
```

---

**In LangChain (message objects):**

LangChain wraps those into **message classes**, like:

```
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, how are you?"),
    AIMessage(content="I'm doing well! How can I assist you today?"),
    HumanMessage(content="What is quantum computing?")
]
```

The format is very similar, we're just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

Then you pass them to the model:

```
response = chat.invoke(messages)
print(response.content)
```

In [21]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

We generate the next response from the AI by passing these messages to the `ChatGroq` object.

Like saying to the AI:

“Here’s what has been said so far — now tell me what the AI should say next.”

LangChain then handles formatting and sending this to the LLM backend, and res stores the AI’s next reply.

**In Short:**
* You define a conversation (via messages).
* Call the LLM using chat(messages).
* Get a response back — stored in res.

In [22]:
res = chat(messages)

In [11]:
res

NameError: name 'res' is not defined

To see the models reply

In [23]:
print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity, two theories that are known to be incompatible within the framework of classical physics. It's a complex and mind-bending concept, but I'll try to break it down for you in simple terms.

**What is String Theory?**

String theory posits that the fundamental building blocks of the universe are not particles, but tiny, vibrating strings. These strings are too small to be seen, but they vibrate at different frequencies, giving rise to the various particles we observe in the universe, such as electrons, photons, and quarks.

**The Three Main Assumptions of String Theory:**

1. **Strings are the fundamental objects**: Instead of particles being the fundamental units of the universe, strings are the basic building blocks.
2. **Strings vibrate at different frequencies**: The different vibrational modes of the strings give rise to the various particles we observe in the unive

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [24]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory can produce a unified theory for several reasons:

**1. Unification of Forces:**

String theory attempts to unify the fundamental forces of nature, including:

* **Gravity**: Described by general relativity
* **Electromagnetism**: Described by quantum electrodynamics (QED)
* **Weak Nuclear Force**: Described by the electroweak theory
* **Strong Nuclear Force**: Described by quantum chromodynamics (QCD)

String theory provides a framework for unifying these forces, potentially explaining why they are distinct at low energies but become unified at high energies.

**2. Grand Unified Theories (GUTs):**

String theory can be thought of as a GUT, where the fundamental forces are unified at a high-energy scale. This is achieved through the existence of extra dimensions, which allow for the unification of forces that would otherwise be separate.

**3. Supersymmetry:**

String theory requires the existence of supersymmetry (SUSY), which proposes that every 

## Dealing with Hallucinations

We have our chatbot, but as mentioned — the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about Deepseek R1.

In [25]:
print(res.content)

Physicists believe that string theory can produce a unified theory for several reasons:

**1. Unification of Forces:**

String theory attempts to unify the fundamental forces of nature, including:

* **Gravity**: Described by general relativity
* **Electromagnetism**: Described by quantum electrodynamics (QED)
* **Weak Nuclear Force**: Described by the electroweak theory
* **Strong Nuclear Force**: Described by quantum chromodynamics (QCD)

String theory provides a framework for unifying these forces, potentially explaining why they are distinct at low energies but become unified at high energies.

**2. Grand Unified Theories (GUTs):**

String theory can be thought of as a GUT, where the fundamental forces are unified at a high-energy scale. This is achieved through the existence of extra dimensions, which allow for the unification of forces that would otherwise be separate.

**3. Supersymmetry:**

String theory requires the existence of supersymmetry (SUSY), which proposes that every 

Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

## Alternate Way : Source Knowledge

There is another way of feeding knowledge into LLMs. It is called _source knowledge_ and it refers to any information fed into the LLM via the prompt. We can try that with the Deepseek question. We can take the paper abstract from the [Deepseek R1 paper](https://arxiv.org/abs/2501.12948).

In [26]:
source_knowledge = (
    "We introduce our first-generation reasoning models, DeepSeek-R1-Zero and "
    "DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale "
    "reinforcement learning (RL) without supervised fine-tuning (SFT) as a "
    "preliminary step, demonstrates remarkable reasoning capabilities. Through "
    "RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and "
    "intriguing reasoning behaviors. However, it encounters challenges such as "
    "poor readability, and language mixing. To address these issues and "
    "further enhance reasoning performance, we introduce DeepSeek-R1, which "
    "incorporates multi-stage training and cold-start data before RL. "
    "DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on "
    "reasoning tasks. To support the research community, we open-source "
    "DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, "
    "32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama."
)

We can feed this additional knowledge into our prompt with some instructions telling the LLM how we'd like it to use this information alongside our original query.

In [27]:
query = "What is so special about Deepseek R1?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

Now we feed this into our chatbot as we were before.

In [28]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [29]:
print(res.content)

However, I think there's been a misunderstanding. The context provided doesn't mention anything about string theory or a unified theory. Instead, it appears to be a discussion about a machine learning model called DeepSeek-R1 and its variants.

If you'd like to ask about what makes DeepSeek-R1 special, I can try to answer based on the provided context:

DeepSeek-R1 is special because it:

1. Incorporates multi-stage training, which is a more sophisticated approach to training machine learning models.
2. Utilizes cold-start data before reinforcement learning (RL), which can help improve the model's performance.
3. Achieves performance comparable to OpenAI-o1-1217 on reasoning tasks, indicating its ability to reason effectively.
4. Is open-sourced, making it available for the research community to use and build upon.
5. Has several dense models distilled from it, which can be used for different applications.

However, if you'd like to discuss the concept of a unified theory in the contex

## How do we get this information in the first place?

The quality of this answer is phenomenal. This is made possible thanks to the idea of augmented our query with external knowledge (source knowledge). There's just one problem

This is where Pinecone and vector databases comes in place, as they can help us here too. But first, we'll need a dataset.

## Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the `"jamescalam/deepseek-r1-paper-chunked"` dataset. This dataset contains the Deepseek R1 paper pre-processed into RAG-ready chunks.

In [30]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/deepseek-r1-paper-chunked",
    split="train"
)

dataset

C:\Users\shama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\shama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shama\.cache\huggingface\hub\datasets--jamescalam--deepseek-r1-paper-chunked. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.c

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'num_tokens', 'pages', 'source'],
    num_rows: 76
})

In [31]:
dataset[0]

{'doi': '2501.12948v1',
 'chunk-id': 1,
 'chunk': "uestion: If a > 1, then the sum of the real solutions of √a - √a + x = x is equal to Response: <think> To solve the equation √a – √a + x = x, let's start by squaring both . . . (√a-√a+x)² = x² ⇒ a - √a + x = x². Rearrange to isolate the inner square root term:(a – x²)² = a + x ⇒ a² – 2ax² + (x²)² = a + x ⇒ x⁴ - 2ax² - x + (a² – a) = 0",
 'num_tokens': 145,
 'pages': [1],
 'source': 'https://arxiv.org/abs/2501.12948'}

## Dataset Overview

The dataset we are using is sourced from the Deepseek R1 ArXiv papers. Each entry in the dataset represents a "chunk" of text from the R1 paper.

Because most **L**arge **L**anguage **M**odels (LLMs) only contain knowledge of the world as it was during training, even many of the newest LLMs cannot answer questions about Deepseek R1 — at least not without this data.

## Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our Pinecone client, this requires a [free API key](https://app.pinecone.io).

In [50]:
from pinecone import Pinecone

# initialize client
pc = Pinecone(api_key=pinecone_key)

Delete the old one to save the resources

In [51]:
index_name = "rag1"
print("Existing indexes:", pc.list_indexes())



Existing indexes: [{
    "name": "rag1",
    "metric": "dotproduct",
    "host": "rag1-vzu7ytp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [35]:
from pinecone import ServerlessSpec, CloudProvider, AwsRegion, Metric

pc.create_index(
    name=index_name,
    metric=Metric.DOTPRODUCT,
    dimension=384,  # ✅ match your embedding model
    spec=ServerlessSpec(
        cloud=CloudProvider.AWS,
        region=AwsRegion.US_EAST_1
    )
)

{
    "name": "rag1",
    "metric": "dotproduct",
    "host": "rag1-vzu7ytp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

Our index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings we will HuggingFace's `sentence-transformers/all-MiniLM-L6-v2` model — we can access it via LangChain like so:

In [52]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Using this model we can create embeddings like so:

In [53]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 384)

From this we get two (aligning to our two chunks of text) CHANGE-dimensional embeddings.

We're now ready to embed and index all our our data! We do this by looping through our dataset and embedding and inserting everything in batches.

In [ ]:
from pinecone import Pinecone

# Replace this with your real Pinecone key (e.g., pcsk_....)
pc = Pinecone(api_key="")

# Index name must match exactly what’s created in your Pinecone console
index_name = "rag1"

index = pc.Index(index_name)


In [56]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

100%|██████████| 1/1 [00:06<00:00,  6.87s/it]


We can check that the vector index has been populated using `describe_index_stats` like before:

In [57]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {'': {'vector_count': 76}},
 'total_vector_count': 76,
 'vector_type': 'dense'}

# Retrieval Augmented Generation

We've built a fully-fledged knowledge base. Now it's time to link that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.

In [58]:
from langchain_pinecone import PineconeVectorStore

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,
    text_key=text_field
)

Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Llama 2.

In [59]:
query = "What is so special about Deepseek R1?"

vectorstore.similarity_search(query, k=3)

[Document(id='2501.12948v1-39', metadata={'source': 'https://arxiv.org/abs/2501.12948'}, page_content='## 1.2. Summary of Evaluation Results - **Reasoning tasks:** (1) DeepSeek-R1 achieves a score of 79.8% Pass@1 on AIME 2024, slightly surpassing OpenAI-01-1217. On MATH-500, it attains an impressive score of 97.3%, performing on par with OpenAI-01-1217 and significantly outperforming other models. (2) On coding-related tasks, DeepSeek-R1 demonstrates expert level in code competition tasks, as it achieves 2,029 Elo rating on Codeforces outperforming 96.3% human participants in the competition. For engineering-related tasks, DeepSeek-R1 performs slightly better than DeepSeek-V3, which could help developers in real world tasks. - **Knowledge:** On benchmarks such as MMLU, MMLU-Pro, and GPQA Diamond, DeepSeek-R1 achieves outstanding results, significantly outperforming DeepSeek-V3 with scores of 90.8% on MMLU, 84.0% on MMLU-Pro, and 71.5% on GPQA Diamond. While its performance is slightly 

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to link the output from our `vectorstore` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [60]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [61]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    ## 1.2. Summary of Evaluation Results - **Reasoning tasks:** (1) DeepSeek-R1 achieves a score of 79.8% Pass@1 on AIME 2024, slightly surpassing OpenAI-01-1217. On MATH-500, it attains an impressive score of 97.3%, performing on par with OpenAI-01-1217 and significantly outperforming other models. (2) On coding-related tasks, DeepSeek-R1 demonstrates expert level in code competition tasks, as it achieves 2,029 Elo rating on Codeforces outperforming 96.3% human participants in the competition. For engineering-related tasks, DeepSeek-R1 performs slightly better than DeepSeek-V3, which could help developers in real world tasks. - **Knowledge:** On benchmarks such as MMLU, MMLU-Pro, and GPQA Diamond, DeepSeek-R1 achieves outstanding results, significantly outperforming DeepSeek-V3 with scores of 90.8% on MMLU, 84.0% on MMLU-Pro, and 71.5% on GPQA Diamond. While its performance is slightly below that of OpenAI-01-1217 on these be

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [62]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

DeepSeek-R1 is a highly advanced artificial intelligence (AI) model that has demonstrated exceptional performance in various reasoning-related tasks. What makes it special is its ability to excel in a wide range of benchmarks, including:

1. **High accuracy**: DeepSeek-R1 achieves high accuracy scores in tasks such as AIME 2024, Codeforces, GPQA Diamond, MATH-500, and MMLU, with some models reaching scores as high as 96.3% and 96.6%.
2. **Competitive performance**: DeepSeek-R1 models consistently rank high across multiple benchmarks, often outperforming other leading models such as OpenAI-01-1217 and Claude-3.5-Sonnet-1022.
3. **Robust performance**: DeepSeek-R1 demonstrates robust performance across different tasks and accuracy levels, with some models maintaining competitive scores even at lower accuracy levels (60 and 40).
4. **Expert-level performance in coding-related tasks**: DeepSeek-R1 demonstrates expert-level performance in code competition tasks, achieving a 2,029 Elo rating

We can continue with another Deepseek R1:

In [63]:
prompt = HumanMessage(
    content=augment_prompt(
        "how does deepseek r1 compare to deepseek r1 zero?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Unfortunately, the contexts provided do not contain any information about DeepSeek-R1-Zero. They only provide information about DeepSeek-R1 and its performance on various benchmarks. Therefore, I cannot provide a comparison between DeepSeek-R1 and DeepSeek-R1-Zero.

However, I can suggest that DeepSeek-R1-Zero might be an earlier version or a precursor to DeepSeek-R1, as it is mentioned in the first context as a model that demonstrates "remarkable reasoning capabilities" but also has some limitations such as poor readability and language mixing. If you have any more information about DeepSeek-R1-Zero, I may be able to provide a more informed comparison.


You can continue asking questions about Deepseek R1, but once you're done you can delete the index to save resources:

In [64]:
pc.delete_index(index_name)